In [1]:
%config InlineBackend.figure_formats = ['svg']

In [2]:
import sympy
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# Python 神经网络

## 参考

- <https://github.com/makeyourownneuralnetwork/makeyourownneuralnetwork>


In [3]:
class Layer(object):

    def forward(self, x):
        pass

    def backward(self, dy):
        pass

    def update(self, lr=0.1):
        pass

## 线性层

In [4]:
x, w = sympy.symbols("x, w")
x, w

(x, w)

In [5]:
dx = sympy.diff(x * w, x)
dx

w

In [6]:
class LinearLayer(Layer):

    def __init__(self, input, output) -> None:
        self.w = np.random.normal(
            loc=0.0,
            scale=pow(input, -0.5),
            size=(input, output)
        )

    def forward(self, x):
        self.x = x
        y = np.dot(x, self.w)
        return y

    def backward(self, dy):
        dx = np.dot(dy, self.w.T)
        self.dw = np.dot(self.x.reshape(-1, 1), dy.reshape(1, -1))
        return dx

    def update(self, lr=0.1):
        self.w += lr * self.dw

In [7]:
x = np.array([0.3, 0.5])
dy = np.array([0.1, 0.2, -0.3])
layer = LinearLayer(2, 3)
y = layer.forward(x)
print("y:", y)

dx = layer.backward(dy)
print("dx:", dx)
print("dw:", layer.dw)

y: [-0.3932558   0.18136254 -0.0159218 ]
dx: [-0.10257795  0.0649937 ]
dw: [[ 0.03  0.06 -0.09]
 [ 0.05  0.1  -0.15]]


## Sigmoid 层

In [8]:
x = sympy.symbols("x")
y = 1 / (1 + sympy.exp(-x))
sympy.Eq(sympy.Symbol('y'), y)

Eq(y, 1/(1 + exp(-x)))

In [9]:
sympy.diff(y, x)

exp(-x)/(1 + exp(-x))**2

$
\begin{aligned}
& e^{-x} \over (1 + e^{-x})^2  \\
=&  {1 \over 1 + e^{-x}}  \cdot {e^{-x} \over 1 + e^{-x} } \\
=&  {1 \over 1 + e^{-x}}  \cdot {1 + e^{-x} - 1 \over 1 + e^{-x} } \\
=&  {1 \over 1 + e^{-x}}  \cdot \left(1 -  {1 \over 1 + e^{-x}}\right) \\
=& y * (1 - y)
\end{aligned}
$


In [10]:
class SigmoidLayer(Layer):

    def forward(self, x):
        # scipy.special.expit(x)
        self.y = 1 / (1 + np.exp(-x))
        return self.y

    def backward(self, dy):
        dx = dy * (1.0 - self.y) * self.y
        return dx

In [11]:
layer1 = LinearLayer(2, 3)
layer2 = SigmoidLayer()

In [12]:
x = np.array([0.3, 0.5])
t = np.array([0.0, 0.0, 1.0])

epoch = 100000
bar = tqdm(range(epoch))

for _ in bar:
    y = layer1.forward(x)
    y = layer2.forward(y)

    dy = t - y

    dx = layer2.backward(dy)
    layer1.backward(dx)
    layer1.update()

y = layer1.forward(x)
y = layer2.forward(y)
y

100%|██████████| 100000/100000 [00:01<00:00, 53775.98it/s]


array([0.01244283, 0.01244231, 0.98755844])

In [13]:
class NeuralNetwork(object):

    def __init__(self, input, hidden, output) -> None:
        self.layers = [
            LinearLayer(input, hidden),
            SigmoidLayer(),
            LinearLayer(hidden, output),
            SigmoidLayer(),
        ]

    def forward(self, x: np.ndarray):
        for layer in self.layers:
            x = layer.forward(x)
        return x

    def backward(self, dy):
        for layer in self.layers[::-1]:
            dy = layer.backward(dy)
        return dy

    def update(self, lr=0.1):
        for layer in self.layers[::-1]:
            layer.update(lr)

In [14]:
net = NeuralNetwork(2, 30, 2)

x_batch = np.array([
    [0.0, 0.0],
    [0.0, 1.0],
    [1.0, 0.0],
    [1.0, 1.0],
])
t_batch = np.array([
    [0.99, 0.01],
    [0.01, 0.99],
    [0.01, 0.99],
    [0.99, 0.01],
])

for i in range(4):
    y = net.forward(x_batch[i])
    print(x_batch[i], y, np.argmax(y), np.argmax(t_batch[i]))

epoch = 100000
bar = tqdm(range(epoch))
for _ in bar:
    for i in range(4):
        y = net.forward(x_batch[i])
        dy = t_batch[i] - y
        # loss = sum(dy ** 2)
        dx = net.backward(dy)
        net.update()
        # bar.set_postfix(dict(loss=f"{loss}"))

for i in range(4):
    y = net.forward(x_batch[i])
    print(x_batch[i], y, np.argmax(y), np.argmax(t_batch[i]))

[0. 0.] [0.47576226 0.53161721] 1 0
[0. 1.] [0.44784662 0.56425225] 1 1
[1. 0.] [0.48221704 0.49349152] 1 1
[1. 1.] [0.46289581 0.52574148] 1 0


100%|██████████| 100000/100000 [00:14<00:00, 7028.96it/s]

[0. 0.] [0.98397524 0.01598661] 0 0
[0. 1.] [0.01572814 0.98430291] 1 1
[1. 0.] [0.01586148 0.98415649] 1 1
[1. 1.] [0.98436763 0.01562395] 0 0
